# T5 style transfer model

1. Run all cells
2. Scroll down to the interactive part to explore T5

In [1]:
# load model and config from public folder https://drive.google.com/drive/folders/1XOWxF54WWqIHRbWQ0YUDf7qDt3LbIrsE?usp=sharing
! echo "Prepare download of T5 model..."
! mkdir t5_model_2
%cd t5_model_2
! echo "download config of model"
! gdown "https://drive.google.com/uc?id=1RrgKJAXK3lujRKSfCgo7CA2bqvOQUhup"
! echo "download model"
! gdown "https://drive.google.com/uc?id=1RXLJgNpjBKMmc5-F781NZBmxRV9WJ2xM"
%cd ..

# set path to T5 model
best_model_path = "/content/t5_model_2/"

# install libraries
! echo "installing libraries..."
!pip install -q sentencepiece
!pip install -q torch==1.4.0
!pip install -q transformers==2.9.0
!pip install -q pytorch_lightning==0.7.5

Prepare download of T5 model...
/content/t5_model_2
download config of model
Downloading...
From: https://drive.google.com/uc?id=1RrgKJAXK3lujRKSfCgo7CA2bqvOQUhup
To: /content/t5_model_2/config.json
100% 1.21k/1.21k [00:00<00:00, 2.08MB/s]
download model
Downloading...
From: https://drive.google.com/uc?id=1RXLJgNpjBKMmc5-F781NZBmxRV9WJ2xM
To: /content/t5_model_2/pytorch_model.bin
2.95GB [00:42, 69.1MB/s]
/content
installing libraries...
     |████████████████████████████████| 1.2MB 9.4MB/s 
     |████████████████████████████████| 753.4MB 24kB/s 
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.4.0 which is incompatible.
     |████████████████████████████████| 645kB 9.1MB/s 
     |████████████████████████████████| 5.6MB 18.5MB/s 
     |████████████████████████████████| 901kB 52.8MB/s 
     |████████████████████████████████| 235kB 7.3MB/s 
     |██

In [2]:
## prepare for model execution

import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

def set_seed(seed):
  torch.manual_seed(seed)
#  if torch.cuda.is_available():
#   torch.cuda.manual_seed_all(seed)

def infer(sentence="", n_sentences=1):
  text =  "paraphrase: " + sentence + " </s>"

  max_len = 256

  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 5
  beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=256,
      top_k=120,
      top_p=0.95,
      early_stopping=True,
      num_return_sequences=n_sentences
  )

  final_outputs =[]
  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if sent.lower() != sentence.lower() and sent not in final_outputs:
          final_outputs.append(sent)
  return final_outputs


set_seed(42)

print("load model...")
model = T5ForConditionalGeneration.from_pretrained(best_model_path)
tokenizer = T5Tokenizer.from_pretrained('t5-large')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

load model...



device  cuda


In [8]:
#@title Interactive

#@markdown Please set your sentences, that you would like to be transferred.
text = "I am very happy for this model." #@param {type: "string"}

#@markdown How many sentences do you want to be generated?
n_sentences = 5 #@param {type:"slider", min:1, max:5, step:1}

print("\n".join(infer(text, n_sentences)))

this is a very nice model.
this model is very nice.
this model is a great joy to me.
this model is very well-defined.
i am very happy with this model.
